In [ ]:
# don't run this unless you're running SRM
#!pip install brainiak

In [9]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
#from brainiak.funcalign.srm import SRM

In [23]:
roi = 'A1'
filepath = 'drive/MyDrive/fMRI_music_data/%s_by_subject/'%roi
subjects = ['03','15','20','23']

In [24]:
# load data from one subject to inspect
s = '03'
test = np.loadtxt(filepath+'%s_s1%s.txt'%(roi,s))
print(test.shape)

(516, 2775)


There are 516 voxels in A1. The first three entries of each row are voxel coordinates. Then there are 18 runs of 154 TRs each. The first six TRs are zeros.

First, load in the data from all subjects for processing.

In [34]:
data = []

# load in the data from each subjects and append it to the list
for subj in subjects: data.append(np.loadtxt(filepath+'%s_s1%s.txt'%(roi,subj)))

# check the size of the list and its elements
print(len(data))
print(data[0].shape)

# recast the list into an array and check the shape
data = np.asarray(data)
print(data.shape)

4
(516, 2775)
(4, 516, 2775)


Process the data to remove unnecessary elements and runs. We're not using the control conditions for this analysis so we only keep the first twelve runs (the runs with scrambled conditions). 

In [35]:
# crop the first three entries of each row (voxel coordinates)
data = data[:,:,3:]
print('without voxel coords: ', data.shape)

# reshape into runs
data = np.reshape(data,(4,516,18,154))

# crop the six empty TRs at the beginning of each run
# drop the control conditions
data = data[:,:,:12,6:]
print('final data shape: ', data.shape)

without voxel coords:  (4, 516, 2772)
final data shape:  (4, 516, 12, 148)


For this analysis, we're interested in event segmentation in run 1 of the 8B condition. Since subjects played the conditions in a counterbalanced order, the run of interest is in a different location in each subject's data. Which run is 8B run 1 is recorded in the 'loc' variable.

In [6]:
loc = [0,1,2,2]

For each subject, copy the data to a temporary variable (`this_data`). Take the run of interest and append it to `test_data` then delete it from `this_data`. Then, take the rest of that subject's data and append it to `train_data`.


In [40]:
train_data = []
test_data = []

for s in range(len(subjects)):
  subj = subjects[s]
  this_data = np.copy(data[s,:,:,:])

  # append the run of interest to test_data
  test_data.append(this_data[:,loc[s],:])

  # delete the run from this_data
  this_data = np.delete(this_data,loc[s],1)

  # append the rest of the data to train_data
  train_data.append(this_data)

In [43]:
# recast our lists into arrays
train_data = np.array(train_data)
test_data = np.array(test_data)

# sanity check
print(train_data.shape)
print(test_data.shape)

(4, 516, 11, 148)
(4, 516, 148)
